# Minimal Model Reasoning with Machine Learning Models
Code Implementation for Odyssey Project.

Fork of: https://github.com/porscheofficial/sls_sat_solving_with_deep_learning/blob/main/python/src/evaluate_with_given_params.py

Code adapted to use the GNN pre-trained Deep Learning Models as a SAT Solver, in order to Solve for Minimal Models.

# Details of Pre-Trained Models from Original Repository

In total, we provide three pre-trained models for the 3-SAT dataset and six pre-trained models for the pseudo-industrial datasets. They found in the folders `../../Data/models/random_3SAT/` and `../../Data/models/pseudo-industrial/`, respectively. The following table summarizes the characteristics of the models.

| Path of the pre-trained model        | Description of model                                                                                                    |
|----------------------------------|-----------------------------------------------------------------------------------------------------------------|
| `../../Data/models/random_3SAT/3_SAT_LLL.npy`| model trained on random 3-SAT using only the LLL-Loss     |
| `../../Data/models/random_3SAT/3_SAT_Gibbs.npy`|  model trained on random 3-SAT using only the Gibbs-Loss |
| `../../Data/models/random_3SAT/3_SAT_Gibbs_LLL.npy`|model trained on random 3-SAT using both the LLL-Loss and the Gibbs-Loss |
| `../../Data/models/pseudo_industrial/g4sat_easy_ca.npy`| model trained on easy CA instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_medium_ca.npy`| model trained on medium CA instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_hard_ca.npy`| model trained on hard CA instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_easy_ps.npy`| model trained on easy PS instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_medium_ps.npy`| model trained on medium PS instances using only the Gibbs-Loss     |
| `../../Data/models/pseudo_industrial/g4sat_hard_ps.npy`| model trained on hard PS instances using only the Gibbs-Loss     |

The corresponding hyperparameters used for the training are specified in the corresponding config files in `../../experiments/configs/`.

In [1]:
# Ensure that both paths are the same (change jupyter notebook's kernel if needed)
import sys
print(sys.executable)
!which python

/home/elija/miniconda3/envs/SLSenv/bin/python
/home/elija/miniconda3/envs/SLSenv/bin/python


In [2]:
import os

def rename_dat_to_cnf(testpath, size_limit):
    counter = 0
    for file in os.listdir(testpath): 
        if file.endswith(".dat"):
            original_file = os.path.join(testpath, file)
            file_size = os.path.getsize(original_file)
            
            if (file_size < size_limit): # 2048, 51200, 102400 less than 100KB (1024 x 100 bytes)
                cnf_name = os.path.splitext(file)[0]
                new_file = os.path.join(testpath, f"{cnf_name}.cnf")
                print()
                print(f"{original_file}: {file_size} bytes")
                print(new_file)
                os.rename(original_file, new_file)
                counter += 1
                
    print(f"No. of Files Renamed: {counter}")

In [3]:
import os
from data_utils import create_solutions_from_cnf

def prepare_cnf_data(testpath):
    # Create _sol.pkl files for each CNF file
    # check that function has been modified to maintain _sat.pkl naming convention regardless of SAT/UNSAT
    create_solutions_from_cnf(testpath)

    # Create a directory for each CNF file.
    # Move the CNF and respective _sol.pkl file into the respective directories.
    for file in os.listdir(testpath): 
        if file.endswith(".cnf"):
            print()
            print(os.path.join(testpath, file))
            
            cnf_name = os.path.splitext(file)[0]  # take only file name (without .cnf extension)
            cnf_folder = os.path.join(testpath, cnf_name)
            os.makedirs(cnf_folder, exist_ok=True)
            
            print(f"Created folder: {cnf_folder}")
            
            # Full original paths
            cnf_path = os.path.join(testpath, f"{cnf_name}.cnf")
            sol_path = os.path.join(testpath, f"{cnf_name}_sol.pkl")
            #print(cnf_path)
            #print(sol_path)

            os.rename(cnf_path, os.path.join(cnf_folder, f"{cnf_name}.cnf"))
            os.rename(sol_path, os.path.join(cnf_folder, f"{cnf_name}_sol.pkl"))
            
            print(f"Move CNF and solution files for {cnf_name} into {cnf_folder}")
    

In [4]:
import os
import numpy as np
from evaluate_with_given_params import load_model_and_test

# Solve with pre-trained GNN model
def solve_with_trained_model(cnf_file_path, model_path):
    print("Loading Model and Test...")
    results = load_model_and_test(
        data_path=cnf_file_path, # Directory containing CNF file(s)
        model_path=model_path,   # Pre-trained model to use
        n_steps=100,             # No. of steps
        n_runs=10,               # No. of runs
        algo_type="moser",       # "moser" for Moser-Tardos (MT) algorithm or "probsat" for WalkSAT algorithm
        path_save=False, 
        keep_traj=True,     
        pre_compute_mapping=True,
        prob_flip_best=0
    )
    
    # Interpret Results
    model_path, model_details, n_array, alpha_array, energies_array_mean, energies_array_median, total_steps = results
    
    # For Debugging
    #print(*results)
    #print(len(energies_array_mean))
    #print(*energies_array_mean)
    
    print(f"No. of Variables: {n_array[0]}")
    print(f"Total Steps: {total_steps}")
    print(f"Last Step's Mean Energy: {energies_array_mean[-1]}")
    
    # Run x steps to find solution, take result from average energies of the last step (of 10 runs)
    # No. of Runs affect last array (no. of steps taken of each run). Mean energies is averaged out of all runs
    if energies_array_mean[-1] == 0:
        # Formula SATISFIABLE (can be solved)
        print("SAT")
        return True
    else:
        # Formula may be UNSATISFIABLE (cannot be solved) or needs more steps
        print("UNSAT / may require more steps")
        return False
        
    #return results

In [5]:
from pysat.solvers import Solver, Minisat22
from pysat.formula import CNF

def compute_minimal_model(F):
    # Make mutable copy of F
    #print(F.clauses)
    working_formula = CNF(from_clauses = F.clauses)
    #print(working_formula)

    while True:
        with Minisat22(bootstrap_with = working_formula.clauses) as solver: # solver deleted after end of with block
            #print(solver.solve())
            if not (solver.solve()):
                return "UNSAT"

            model = solver.get_model()
            #print(model)
            model_set = set(model)
            #print(model_set)

            # If model is minimal, return the minimal model.
            if is_minimal(F, model_set):
                return model_set

            # Otherwise, model is not minimal
            # Block the current non-minimal model and try again
            # Add blocking clause to avoid repeating this model
            block_clause = block_model(model_set)
            print("Block Clause: ", block_clause)
            working_formula.append(block_clause)

def is_minimal(F, model_set):
    true_vars = set()
    false_vars = set()

    for lit in model_set:
        if lit > 0: # if literal is true
            true_vars.add(lit)
        else: # else literal is false
            false_vars.add(lit)

    #print("Model Set: ")
    #print(*model_set)
    #print("True Vars: ")
    #print(*true_vars)
    #print("False Vars: ")
    #print(*false_vars)
    
    # Make mutable copy of F
    minimal_check_formula = CNF(from_clauses = F.clauses)

    # Add unit clause for each false assignment (fix as false)
    for lit in false_vars:
        minimal_check_formula.append([lit])

    # Add disjunction to drop at least one true variable (change to false)
    if true_vars: # set is non-empty
        new_clause = [-v for v in true_vars]
        #print("New Clause: ", new_clause)
        minimal_check_formula.append(new_clause) # new_clause is a disjunction (OR) of literals
        
    else: # trivially minimal if no true variables (empty set is cast as boolean to false)
        return True
    
    #print("Minimal Check Formula: ")
    #print(*minimal_check_formula)
    
    # Traditional SAT Solver (to replace with Deep Learning Model)
    #solver = Minisat22(bootstrap_with = minimal_check_formula.clauses) 
    #result = solver.solve()
    #solver.delete()
    
    # Deep Learning SLS SAT Solver
    minimal_check_formula.to_file("../../Data/random_sat_data/extracted/output/minimal_check_formula.cnf")
    prepare_cnf_data("../../Data/random_sat_data/extracted/output/")
    
    print("\n INVOKING DEEP LEARNING SAT SOLVER \n ")
    # Change model accordingly 
    result = solve_with_trained_model(cnf_file_path="../../Data/random_sat_data/extracted/output/minimal_check_formula", 
                                      model_path="../../Data/models/pseudo_industrial/g4sat_hard_ps.npy")
    print("\n END OF INVOKING DEEP LEARNING SAT SOLVER \n ")
    print("Result from Deep Learning (is SAT): " + str(result))
    print("Result from Deep Learning (is minimal): " + str(not result) + "\n")
    
    # If result is true (SAT), then F is not minimal
    # Else result is false (UNSAT), then F is minimal.
    return not result

def block_model(model_set):
    # Only negate the literals assigned True (do not include negation of literals assigned False)
    # This blocks the model and all its supersets.
    clause = [-lit for lit in model_set if lit > 0]
    #print("Clause: ", clause)
    return clause

In [6]:
# Examples (where variables a=1, b=2, c=3, d=4, e=5)

# F = (a ∨ b ∨ c), (¬a ∨ ¬b ∨ d), (¬a ∨ ¬b ∨ e)
cnf = CNF() # Conjunctive Normal Form
cnf.append([1, 2, 3])      # a ∨ b ∨ c
cnf.append([-1, -2, 4])    # ¬a ∨ ¬b ∨ d
cnf.append([-1, -2, 5])    # ¬a ∨ ¬b ∨ e
#cnf.append([6, 2, 3])    # f v b v c
minimal_model = compute_minimal_model(cnf)
print("Minimal model 1: ", minimal_model, "\n")
#cnf.to_file("../../Data/random_sat_data/mytest/example1.cnf")

# F = (¬a ∨ ¬b ∨ d), (¬c), (¬d)
cnf2 = CNF() # Conjunctive Normal Form
cnf2.append([-1, -2])
cnf2.append([-3])
cnf2.append([-4])
#print(*cnf2)
minimal_model2 = compute_minimal_model(cnf2)
print("Minimal model 2: ", minimal_model2, "\n")
#cnf2.to_file("../../Data/random_sat_data/mytest/example2.cnf")

# F = (a) AND (¬a)
cnf3 = CNF() # Conjunctive Normal Form
cnf3.append([1])      # a
cnf3.append([-1])    # ¬a
minimal_model3 = compute_minimal_model(cnf3)
print("Minimal model 3: ", minimal_model3, "\n")
#cnf3.to_file("../../Data/random_sat_data/mytest/example3.cnf")
    
print("==========================================")
print("PSEUDO-INDUSTRIAL CNF Formula Testing...")
print("==========================================")

# For each CNF file (CNF Formula), run the Deep Learning-based Minimal Model Solver.
target_directory = "../../Data/random_sat_data/extracted/"
rename_dat_to_cnf(target_directory, size_limit=2048)

for entry in os.listdir(target_directory):
    full_path = os.path.join(target_directory, entry)

    print("NEW ENTRY")
    if os.path.isfile(full_path) and entry.endswith(".cnf"):
        print(f"Found CNF file: {full_path}")
        print(f"Size: {os.path.getsize(full_path)} bytes")

        # For Minimal Model Solving
        cnf_formula = CNF(from_file=full_path)
        minimal_model = compute_minimal_model(cnf_formula)
        print("Minimal model: ", minimal_model)

    elif os.path.isdir(full_path):
        print(f"Found folder: {full_path}")

    print("=========================================\n\n")

Jax plugin configuration error: Plugin module %s could not be loaded
Traceback (most recent call last):
  File "/home/elija/miniconda3/envs/SLSenv/lib/python3.10/site-packages/jax/_src/xla_bridge.py", line 457, in discover_pjrt_plugins
    plugin_module = importlib.import_module(plugin_module_name)
  File "/home/elija/miniconda3/envs/SLSenv/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/elija/miniconda3/envs/SLSenv/lib/python3.10/site-packages/jax_plugins/xla_cuda1

processing ../../Data/random_sat_data/extracted/output/minimal_check_formula.cnf

../../Data/random_sat_data/extracted/output/minimal_check_formula.cnf
Created folder: ../../Data/random_sat_data/extracted/output/minimal_check_formula
Move CNF and solution files for minimal_check_formula into ../../Data/random_sat_data/extracted/output/minimal_check_formula

 INVOKING DEEP LEARNING SAT SOLVER 
 
Loading Model and Test...
LCG
problem  1 of  1
../../Data/models/pseudo_industrial/g4sat_hard_ps.npy
No. of Variables: 5
Total Steps: [[99 99 99 99 99 99 99 99 99 99]]
Last Step's Mean Energy: 0.175
UNSAT / may require more steps

 END OF INVOKING DEEP LEARNING SAT SOLVER 
 
Result from Deep Learning (is SAT): False
Result from Deep Learning (is minimal): True

Minimal model 1:  {1, -5, -4, -3, -2} 

Minimal model 2:  {-4, -3, -1, -2} 

Minimal model 3:  UNSAT 

PSEUDO-INDUSTRIAL CNF Formula Testing...
No. of Files Renamed: 0
NEW ENTRY
Found folder: ../../Data/random_sat_data/extracted/Archive



../../Data/models/pseudo_industrial/g4sat_hard_ps.npy
No. of Variables: 2302
Total Steps: [[99 99 99 99 99 99 99 99 99 99]]
Last Step's Mean Energy: 0.07415199656504937
UNSAT / may require more steps

 END OF INVOKING DEEP LEARNING SAT SOLVER 
 
Result from Deep Learning (is SAT): False
Result from Deep Learning (is minimal): True

Minimal model:  {6, 9, 17, 19, 33, 35, 36, 40, 41, 45, 46, 48, 49, 50, 51, 71, 72, 73, 74, 94, 95, 125, 126, 230, 241, 493, -2302, -2301, -2300, -2299, -2298, -2297, -2296, -2295, -2294, -2293, -2292, -2291, -2290, -2289, -2288, -2287, -2286, -2285, -2284, -2283, -2282, -2281, -2280, -2279, -2278, -2277, -2276, -2275, -2274, -2273, -2272, -2271, -2270, -2269, -2268, -2267, -2266, -2265, -2264, -2263, -2262, -2261, -2260, -2259, -2258, -2257, -2256, -2255, -2254, -2253, -2252, -2251, -2250, -2249, -2248, -2247, -2246, -2245, -2244, -2243, -2242, -2241, -2240, -2239, -2238, -2237, -2236, -2235, -2234, -2233, -2232, -2231, -2230, -2229, -2228, -2227, -2226, -22

../../Data/models/pseudo_industrial/g4sat_hard_ps.npy
No. of Variables: 2302
Total Steps: [[99 99 99 99 99 99 99 99 99 99]]
Last Step's Mean Energy: 0.07269214255045084
UNSAT / may require more steps

 END OF INVOKING DEEP LEARNING SAT SOLVER 
 
Result from Deep Learning (is SAT): False
Result from Deep Learning (is minimal): True

Minimal model:  {6, 9, 14, 17, 19, 33, 35, 36, 40, 41, 45, 46, 48, 49, 50, 51, 71, 72, 73, 74, 94, 95, 125, 126, 230, 241, 493, -2302, -2301, -2300, -2299, -2298, -2297, -2296, -2295, -2294, -2293, -2292, -2291, -2290, -2289, -2288, -2287, -2286, -2285, -2284, -2283, -2282, -2281, -2280, -2279, -2278, -2277, -2276, -2275, -2274, -2273, -2272, -2271, -2270, -2269, -2268, -2267, -2266, -2265, -2264, -2263, -2262, -2261, -2260, -2259, -2258, -2257, -2256, -2255, -2254, -2253, -2252, -2251, -2250, -2249, -2248, -2247, -2246, -2245, -2244, -2243, -2242, -2241, -2240, -2239, -2238, -2237, -2236, -2235, -2234, -2233, -2232, -2231, -2230, -2229, -2228, -2227, -2226,

LCG
problem  1 of  1
../../Data/models/pseudo_industrial/g4sat_hard_ps.npy
No. of Variables: 2302
Total Steps: [[99 99 99 99 99 99 99 99 99 99]]
Last Step's Mean Energy: 0.07150837988826816
UNSAT / may require more steps

 END OF INVOKING DEEP LEARNING SAT SOLVER 
 
Result from Deep Learning (is SAT): False
Result from Deep Learning (is minimal): True

Minimal model:  {5, 16, 18, 31, 36, 37, 39, 48, 50, 51, 69, 71, 72, 73, 97, 98, 126, 129, 130, 133, 136, 138, 140, 148, 203, 208, 214, 226, 398, 493, -2302, -2301, -2300, -2299, -2298, -2297, -2296, -2295, -2294, -2293, -2292, -2291, -2290, -2289, -2288, -2287, -2286, -2285, -2284, -2283, -2282, -2281, -2280, -2279, -2278, -2277, -2276, -2275, -2274, -2273, -2272, -2271, -2270, -2269, -2268, -2267, -2266, -2265, -2264, -2263, -2262, -2261, -2260, -2259, -2258, -2257, -2256, -2255, -2254, -2253, -2252, -2251, -2250, -2249, -2248, -2247, -2246, -2245, -2244, -2243, -2242, -2241, -2240, -2239, -2238, -2237, -2236, -2235, -2234, -2233, -2232

../../Data/models/pseudo_industrial/g4sat_hard_ps.npy
No. of Variables: 383
Total Steps: [[99 99 99 99 99 99 99 99 99 99]]
Last Step's Mean Energy: 0.004239401496259352
UNSAT / may require more steps

 END OF INVOKING DEEP LEARNING SAT SOLVER 
 
Result from Deep Learning (is SAT): False
Result from Deep Learning (is minimal): True

Minimal model:  {2, 4, 6, 8, 9, 11, 12, 13, 14, 16, 17, 18, 20, 28, 30, 55, 67, 68, 115, 137, 207, 213, 217, 222, 279, -383, -382, -381, -380, -379, -378, -377, -376, -375, -374, -373, -372, -371, -370, -369, -368, -367, -366, -365, -364, -363, -362, -361, -360, -359, -358, -357, -356, -355, -354, -353, -352, -351, -350, -349, -348, -347, -346, -345, -344, -343, -342, -341, -340, -339, -338, -337, -336, -335, -334, -333, -332, -331, -330, -329, -328, -327, -326, -325, -324, -323, -322, -321, -320, -319, -318, -317, -316, -315, -314, -313, -312, -311, -310, -309, -308, -307, -306, -305, -304, -303, -302, -301, -300, -299, -298, -297, -296, -295, -294, -293, -

../../Data/models/pseudo_industrial/g4sat_hard_ps.npy
No. of Variables: 26
Total Steps: [[99 99 99 99 99 99 99 99 99 99]]
Last Step's Mean Energy: 0.0547945205479452
UNSAT / may require more steps

 END OF INVOKING DEEP LEARNING SAT SOLVER 
 
Result from Deep Learning (is SAT): False
Result from Deep Learning (is minimal): True

Minimal model:  {3, 4, 5, 11, 13, 17, -26, -25, -24, -23, -22, -21, -20, -19, -18, -16, -15, -14, -12, -10, -9, -8, -7, -6, -1, -2}


NEW ENTRY
Found CNF file: ../../Data/random_sat_data/extracted/out_cost_matrix_component_nr_2992_size_6_cutoff_10.0.cm.cnf
Size: 42 bytes
processing ../../Data/random_sat_data/extracted/output/minimal_check_formula.cnf

../../Data/random_sat_data/extracted/output/minimal_check_formula.cnf
Created folder: ../../Data/random_sat_data/extracted/output/minimal_check_formula
Move CNF and solution files for minimal_check_formula into ../../Data/random_sat_data/extracted/output/minimal_check_formula

 INVOKING DEEP LEARNING SAT SOLVER 
 